# Leçon 2 — Les Large Language Models (LLMs)

## Objectifs
- Comprendre l'auto-régression, les tokens et l'attention (Transformers)
- Connaître fine-tuning, LoRA/QLoRA, RAG et embeddings
- Bonnes pratiques de prompting et d'évaluation

## Idées clés
- **Transformer** : attention multi-tête + MLP, positions, normalisations
- **Auto-régression** : prédire le prochain token
- **Adaptation** : fine-tuning complet, LoRA, RAG
- **Éval** : exact match, BLEU/ROUGE, human eval, coûts/latence


In [ ]:
# --- Similarité sémantique avec un encodeur de phrases ---
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_sentence(text: str) -> torch.Tensor:
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        out = model(**tokens)
    emb = out.last_hidden_state.mean(dim=1)
    return F.normalize(emb, p=2, dim=1)

corpus = [
    "Qu'est-ce que l'IA générative ?",
    "Comment fonctionne l'apprentissage supervisé ?",
    "Explique le mécanisme d'attention des Transformers.",
]
corpus_embs = torch.cat([embed_sentence(x) for x in corpus], dim=0)
query = "Décris le principe d'attention multi-tête."
query_emb = embed_sentence(query)
sims = (query_emb @ corpus_embs.T).squeeze(0)
best_idx = torch.argmax(sims).item()
print('Question:', query)
print('Meilleure correspondance:', corpus[best_idx], '| score:', float(sims[best_idx]))


> Astuce : Pour un RAG minimal, combinez ces embeddings avec une recherche cosine pour retrouver des passages pertinents puis injectez-les dans le prompt du LLM.